{{ badge }}

# Расчетное задание 3

## Оптимизация ССМО

Выполнил студент гр. 3540901/21501 Лихолетов М.Д.

Руководитель, к.т.н., доц. Сиднев А.Г.

## Задание

**Вариант 142**

Оптимизация одноканальной немарковской сети.

\begin{gather}
    ca_0 = 0.49\\
    \lambda_0 = 24\\
    \mu = 800\\
    \{ сs_j \} = \{ 0.25, 0.25, 0.25, 0.25 \}
\end{gather}

$$
Q = \{ q_{ij} \}_{\substack{i = \overline{0,n} \\ j = \overline{0,n}}} =
\begin{bmatrix}
    0 & 0.1 & 0.1 & 0.5 & 0.3 \\
    0.1 & 0 & 0.1 & 0.3 & 0.5 \\
    0.2 & 0.1 & 0 & 0.5 & 0.2 \\
    0 & 0.2 & 0.5 & 0 & 0.3 \\
    0 & 0.2 & 0.3 & 0.4 & 0.1 \\
\end{bmatrix}
$$

где $n$ – число узлов сети, $ca_0$ – квадрат коэффициента вариации входного протока, $\{ cs_j \}$ – вектор квадратов коэффициентов вариации законов обслуживания в узлах сети, $q_{ij}$ – вероятность перехода заявки на узел $j$ после того, как данную заявку обработал узел $i$, $\mu$ – ожидаемое время обслуживания заявки одним узлом, $\lambda_0$ – ожидаемое время поступления заявки на узел 0.

Пусть заявка из 0-го узла с вероятностью 1 поступает в один из узлов сети. То есть в матрице передач $Q$ все элементы 0-й строки кроме некоторого элемента $l$, равного 1, равны 0, тогда $ca_0 = ca_{0l}$. Если заявка из 0-го узла поступает более, чем в один узел, то и в этом случае предполагается, что $ca_{0l} = ca_0, \, l = \overline{1,n}$.

Требуется минимизировать суммарное число заявок в сети $L(\mu)$ при ограничении на суммарную интенсивность обслуживания в узлах $\mu$:

\begin{gather}
    min L(\mu) = \sum_{j=1}^n L_j\\
    \mu = \sum_j^n \mu_j
\end{gather}

Данная задача относится к классу задач SP2.1/G/S/R. Алгоритм решения задачи состоит из следующих этапов:

1. Поиск вероятного распределения $\mu_j = \mu_j^0$ и вычисление $ca_j$ и $cs_j$
2. Итеративный поиск минимального суммарного числа заявок $L(\mu)$

## 1. Поиск допустимого решения

Скорость поступления заявок в узел $j$ можно рассчитать следующим образом:

$$
\lambda_j = \sum_{i=0}^n q_{ij} \lambda_i
$$

где $lambda_j$ – ожидаемое время поступления заявки на узел $j$.

Зная $Q$ и $\lambda_0$, получается следующая система уравнений:

$$
Q = \{ q_{ij} \}_{\substack{i = \overline{0,n} \\ j = \overline{0,n}}} =
\begin{bmatrix}
    0 & 0.1 & 0.1 & 0.5 & 0.3 \\
    0.1 & 0 & 0.1 & 0.3 & 0.5 \\
    0.2 & 0.1 & 0 & 0.5 & 0.2 \\
    0 & 0.2 & 0.5 & 0 & 0.3 \\
    0 & 0.2 & 0.3 & 0.4 & 0.1 \\
\end{bmatrix}
; \quad
\begin{cases}
    \lambda_0 = 24\\
    \lambda_1 = 0.1 \lambda_0 + 0.1 \lambda_2 + 0.2 \lambda_3 + 0.2 \lambda_4\\
    \lambda_2 = 0.1 \lambda_0 + 0.1 \lambda_1 + 0.5 \lambda_3 + 0.3 \lambda_4\\
    \lambda_3 = 0.5 \lambda_0 + 0.3 \lambda_1 + 0.5 \lambda_2 + 0.4 \lambda_4\\
    \lambda_4 = 0.3 \lambda_0 + 0.5 \lambda_1 + 0.2 \lambda_2 + 0.3 \lambda_3 + 0.1 \lambda_4
\end{cases}
$$

In [38]:
from sympy import Indexed, IndexedBase, Tuple, solve

lambda_j = IndexedBase('lambda_j')
lambda_j_equations = Tuple(
    lambda_j[0] - 24,
    0.1 * lambda_j[0] - 1.0 * lambda_j[1] + 0.1 * lambda_j[2] + 0.2 * lambda_j[3] + 0.2 * lambda_j[4],
    0.1 * lambda_j[0] + 0.1 * lambda_j[1] - 1.0 * lambda_j[2] + 0.5 * lambda_j[3] + 0.3 * lambda_j[4],
    0.5 * lambda_j[0] + 0.3 * lambda_j[1] + 0.5 * lambda_j[2] - 1.0 * lambda_j[3] + 0.4 * lambda_j[4],
    0.3 * lambda_j[0] + 0.5 * lambda_j[1] + 0.2 * lambda_j[2] + 0.3 * lambda_j[3] - 0.9 * lambda_j[4],
)
lambda_j = list(solve(lambda_j_equations, lambda_j_equations.atoms(Indexed)).values())
lambda_j

[24.0000000000000,
 53.5942446043165,
 93.2028776978417,
 113.162589928058,
 96.2071942446043]

Далее требуется найти $ca_j$, для чего требуется найти все $\lambda_{ij} = \lambda_i * q_{ij}$:

In [39]:
%precision % .4f

Q = [
    [0, 0.1, 0.1, 0.5, 0.3],
    [0.1, 0, 0.1, 0.3, 0.5],
    [0.2, 0.1, 0, 0.5, 0.2],
    [0, 0.2, 0.5, 0, 0.3],
    [0, 0.2, 0.3, 0.4, 0.1]
]

lambda_ij = []
for i, q_row in enumerate(Q):
    temp = []
    for q in q_row:
        temp.append(lambda_j[i] * q)
    lambda_ij.append(temp)
lambda_ij

[[0, 2.40000000000000, 2.40000000000000, 12.0000000000000, 7.20000000000000],
 [5.35942446043165, 0, 5.35942446043165, 16.0782733812950, 26.7971223021583],
 [18.6405755395683, 9.32028776978417, 0, 46.6014388489209, 18.6405755395683],
 [0, 22.6325179856115, 56.5812949640288, 0, 33.9487769784173],
 [0, 19.2414388489209, 28.8621582733813, 38.4828776978417, 9.62071942446043]]

Теперь можно найти $ca_j$:

\begin{gather}
ca_j = \frac{\lambda_{0j}}{\lambda_j} ca_{0j} + \sum_{i=1}^n \frac{\lambda_{ij}}{\lambda_j} ca_{ij} = \sum_{i=0}^n \frac{\lambda_{ij}}{\lambda_j} ca_{ij}\\
cd_{ij} = q_{ij} cd_{ij} + 1 - q_{ij}
\end{gather}

In [40]:
from numpy.linalg import inv
from numpy import zeros, eye, matmul

caA = zeros((5, 5))
caB = zeros((5, 1))

for i, l_i in enumerate(lambda_ij):
    for j, l_ij in enumerate(l_i):
        caA[j][i] = l_ij / lambda_j[j] * Q[i][j]
        caB[j] = caB[j] + l_ij / lambda_j[j] * (1 - Q[i][j])
caA = caA - eye(5)
caA[0] = [1, 0, 0, 0, 0]
caB[0][0] = -0.49
caj = [c[0] for c in matmul(inv(caA), -caB)]
caj

[0.49,
 0.993604246075317,
 0.9873695457999723,
 0.9678555042962269,
 0.9836027132501883]

Также требуется вычислить $L_j$ и $P_j$:

\begin{gather}
p_j = \frac{\lambda_j}{\mu_j m_j}\\
L_j(\lambda_j, ca_j, \mu_j, cs_j) = \frac{(\frac{\lambda_j}{\mu_j})^2 (ca_j + cs_j) \times g(\lambda_j, ca_j, \mu_j, cs_j)}{2 \big(1 - \frac{\lambda_j}{\mu_j}\big)} + \frac{\lambda_j}{\mu_j}\\
PI_j(\mu_j) = -V_j \frac{\delta L_j(\mu_j)}{\delta(\mu_j)} = \frac{\lambda_j}{(\lambda_j - \mu_j)^2}$\\

cs_1 = 0.25
\end{gather}

In [41]:
from sympy import exp

mu = [200, 200, 200, 200]
cs = [0.25, 0.25, 0.25, 0.25]


def calculate(fl, fca, fm, fcs):
    Lj = (fl / fm) ** 2 * (fca + fcs) * exp(-2 * (1 - fl / fm) * (1 - fca) ** 2 / (3 * (fl / fm) * (fca + fcs))) / (
            2 * (1 - fl / fm))
    PIj = fl / ((fl - fm) ** 2)
    return Lj, PIj


Lj_list = []
PIj_list = []

for i in range(1, 5):
    Lj_result, PIj_result = calculate(lambda_j[i], caj[i], mu[i - 1], cs[i - 1])
    Lj_list.append(Lj_result)
    PIj_list.append(PIj_result)

L_sum = sum(Lj_list)

In [42]:
PIj_list

[0.00250035603910347,
 0.00817165871968174,
 0.0150068366150819,
 0.00893044386763243]

In [43]:
Lj_list

[0.0609922068648138, 0.251591377039613, 0.448793094111279, 0.274976190279542]

In [44]:
L_sum

1.03635286829525

Воспользуемся следующей формулой:

$$
PI_j(\mu_j) (\lambda_j + \epsilon_j) = max \{ PI_j(\mu_j), j \in J_0 \}
$$

Чем выше загрузка узла $j$, тем больше $PI_j = \frac{-\delta L(\mu_j)}{\delta \mu _j}$

Из результатов расчётов можно понять в каких узлах нужно увеличивать или уменьшать интенсивность входного потока.



Далее следует применить следующий алгоритм:

1. Задать множество доступных для распределения узлов $J_0$, и пустые множества $J_1$ и $J_2$.
2. Найти
   $j_1 = arg \underset{(j)}{min} PI_j(\mu_j)$;
   $j_2 = arg \underset{(j)}{max} PI_j(\mu_j)$;
   - если $j_1 \in J_1$, выполнить $J_0 \leftarrow J_0 - \{ j_1 \}$;
   - если $j_2 \in J_2$, выполнить $J_0 \leftarrow J_0 - \{ j_2 \}$;
   - если $j_1 \notin J_1$ и $j_2 \notin J_2$, выполнить $\mu_{j_1} \leftarrow \mu_{j_1} + \Delta$, $\mu_{j_2} \leftarrow \mu_{j_2} + \Delta$, $J_1 \leftarrow J_1 \lor \{ j_2 \}$ и $J_2 \leftarrow J_2 \lor \{ j_1 \}$.

    Если $j_1 = j_2$ или $J_0$ пустое - останов, иначе повторить 2 шаг.

Возьмем $\Delta$ равное 5.

In [50]:
import pandas
from pandas import DataFrame

pandas.options.display.max_rows = None

sets_formatting = {
    '#': [],
    'J0': [],
    'J1': [],
    'J2': [],
    'Действия': []
}
power_redistribution = {
    '#': [],
    'µ1': [],
    'µ2': [],
    'µ3': [],
    'µ4': [],
    'Действия': []
}
calculated_pi = {
    '#': [],
    'PI1': [],
    'PI2': [],
    'PI3': [],
    'PI4': []
}
calculated_l = {
    '#': [],
    'L1': [],
    'L2': [],
    'L3': [],
    'L4': [],
    'L': []
}

j_set = [
    [1, 2, 3, 4],
    [],
    []
]
mu = [200, 200, 200, 200]
delta = 5
eps = [max(PIj_list) / p - lambda_j[i] for i, p in enumerate(PIj_list)]
i = 1
while len(j_set[0]) > 0:
    sets_formatting['#'].append(i)
    power_redistribution['#'].append(i)
    calculated_pi['#'].append(i)
    calculated_l['#'].append(i)

    sets_formatting['J0'].append(','.join(str(j) for j in j_set[0]))
    sets_formatting['J1'].append(','.join(str(j) for j in j_set[1]))
    sets_formatting['J2'].append(','.join(str(j) for j in j_set[2]))

    power_redistribution['µ1'].append(mu[0])
    power_redistribution['µ2'].append(mu[1])
    power_redistribution['µ3'].append(mu[2])
    power_redistribution['µ4'].append(mu[3])

    l1, pi1 = calculate(lambda_j[1], caj[1], mu[0], cs[0])
    l2, pi2 = calculate(lambda_j[2], caj[2], mu[1], cs[1])
    l3, pi3 = calculate(lambda_j[3], caj[3], mu[2], cs[2])
    l4, pi4 = calculate(lambda_j[4], caj[4], mu[3], cs[3])

    calculated_pi['PI1'].append(pi1)
    calculated_pi['PI2'].append(pi2)
    calculated_pi['PI3'].append(pi3)
    calculated_pi['PI4'].append(pi4)

    calculated_l['L1'].append(l1)
    calculated_l['L2'].append(l2)
    calculated_l['L3'].append(l3)
    calculated_l['L4'].append(l4)
    calculated_l['L'].append(l1 + l2 + l3 + l4)

    pi = [pi1, pi2, pi3, pi4]
    pi_actual = []

    for j in range(len(pi)):
        if j + 1 in j_set[0]:
            pi_actual.append(pi[j])

    pi_min = min(pi_actual)
    pi_min_index = pi.index(pi_min) + 1

    pi_max = max(pi_actual)
    pi_max_index = pi.index(pi_max) + 1

    sets_action = []
    dist_actions = []

    if pi_min_index in j_set[1] and pi_min_index in j_set[0]:
        j_set[0].remove(pi_min_index)
        sets_action.append('J0 ← J0 - {}'.format(pi_min_index))
    if pi_max_index in j_set[2] and pi_max_index in j_set[0]:
        j_set[0].remove(pi_max_index)
        sets_action.append('J0 ← J0 - {}'.format(pi_max_index))
    if pi_min_index not in j_set[1] and pi_max_index not in j_set[2]:
        mu[pi_min_index - 1] -= delta
        mu[pi_max_index - 1] += delta
        dist_actions.append('µ{} − ∆'.format(pi_min_index))
        dist_actions.append('µ{} + ∆'.format(pi_max_index))
        if pi_max_index not in j_set[1]:
            j_set[1].append(pi_max_index)
        if pi_min_index not in j_set[2]:
            j_set[2].append(pi_min_index)
        sets_action.append('J1 ← J1 + {}'.format(pi_max_index))
        sets_action.append('J2 ← J2 + {}'.format(pi_min_index))

    sets_formatting['Действия'].append('; '.join(sets_action))
    power_redistribution['Действия'].append('; '.join(dist_actions))

    if pi_max == pi_min:
        break

    i += 1

 Формирование множеств J:

In [51]:
sets_formatting_table = DataFrame(sets_formatting)
sets_formatting_table.set_index('#')

,J0,J1,J2,Действия
#,,,,
1,"1,2,3,4",,,J1 ← J1 + 3; J2 ← J2 + 1
2,"1,2,3,4",3,1,J1 ← J1 + 3; J2 ← J2 + 1
3,"1,2,3,4",3,1,J1 ← J1 + 3; J2 ← J2 + 1
4,"1,2,3,4",3,1,J1 ← J1 + 3; J2 ← J2 + 1
5,"1,2,3,4",3,1,J1 ← J1 + 3; J2 ← J2 + 1
6,"1,2,3,4",3,1,J1 ← J1 + 3; J2 ← J2 + 1
7,"1,2,3,4",3,1,J1 ← J1 + 4; J2 ← J2 + 1
8,"1,2,3,4","3,4",1,J1 ← J1 + 3; J2 ← J2 + 1
9,"1,2,3,4","3,4",1,J1 ← J1 + 2; J2 ← J2 + 1


Перераспределение мощностей:

In [47]:
power_redistribution_table = DataFrame(power_redistribution)
power_redistribution_table.set_index('#')

,µ1,µ2,µ3,µ4,Действия
#,,,,,
1,200,200,200,200,µ1 − ∆; µ3 + ∆
2,195,200,205,200,µ1 − ∆; µ3 + ∆
3,190,200,210,200,µ1 − ∆; µ3 + ∆
4,185,200,215,200,µ1 − ∆; µ3 + ∆
5,180,200,220,200,µ1 − ∆; µ3 + ∆
6,175,200,225,200,µ1 − ∆; µ3 + ∆
7,170,200,230,200,µ1 − ∆; µ4 + ∆
8,165,200,230,205,µ1 − ∆; µ3 + ∆
9,160,200,235,205,µ1 − ∆; µ2 + ∆


 Пересчитанное $PI_j$:

In [48]:
calculated_pi_table = DataFrame(calculated_pi)
calculated_pi_table.set_index('#')

,PI1,PI2,PI3,PI4
#,,,,
1,0.00250035603910347,0.00817165871968174,0.0150068366150819,0.00893044386763243
2,0.00268030355087513,0.00817165871968174,0.0134172534412168,0.00893044386763243
3,0.00288039974684595,0.00817165871968174,0.0120674788733813,0.00893044386763243
4,0.00310376891844955,0.00817165871968174,0.0109115937123291,0.00893044386763243
5,0.00335416526694698,0.00817165871968174,0.00991416575689348,0.00893044386763243
6,0.00363613168422446,0.00817165871968174,0.00904750175931411,0.00893044386763243
7,0.00395520726791263,0.00817165871968174,0.00828970423191940,0.00893044386763243
8,0.00431820144643845,0.00817165871968174,0.00828970423191940,0.00812843981750131
9,0.00473356031891574,0.00817165871968174,0.00762327455741681,0.00812843981750131


 Пересчитанное $L_j$ и $L$:

In [49]:
calculated_l_table = DataFrame(calculated_l)
calculated_l_table.set_index('#')

,L1,L2,L3,L4,L
#,,,,,
1,0.0609922068648138,0.251591377039613,0.448793094111279,0.274976190279542,1.03635286829525
2,0.0647681785813711,0.251591377039613,0.413998417427742,0.274976190279542,1.00533416332827
3,0.0689093215408312,0.251591377039613,0.383264730389272,0.274976190279542,0.978741619249259
4,0.0734647563992884,0.251591377039613,0.355962834606435,0.274976190279542,0.955995158324879
5,0.0784922346490783,0.251591377039613,0.331584007650169,0.274976190279542,0.936643809618403
6,0.0840600436480149,0.251591377039613,0.309712786495114,0.274976190279542,0.920340397462284
7,0.0902494261252786,0.251591377039613,0.290006774924781,0.274976190279542,0.906823768369215
8,0.0971576810071638,0.251591377039613,0.290006774924781,0.255938146451469,0.894693979423027
9,0.104902176489658,0.251591377039613,0.272181452047316,0.255938146451469,0.884613152028056


## Вывод

В данной работе была произведена оптимизация ССМО, в частности перераспределение мощностей в системе, для уменьшения очереди.

Начальные значения:

$\mu$ = [200, 200, 200, 200]
$L_j$ = [0.0610, 0.2516, 0.4488, 0.2750]
$L$ = 1.0364

После оптимизации:

$\mu$ = [220, 220, 180, 180]
$L_j$ = [0.0488, 0.1926, 0.6479, 0.3785]
$L$ = 1.2678
